In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import operator
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import minkowski_distance, distance
from sklearn.metrics import pairwise

!pip install imdbpy
from imdb import IMDb
ia = IMDb()

     |████████████████████████████████| 298 kB 885 kB/s 


2021-11-11 07:53:33,598 INFO [imdbpy.parser.http.piculet] /opt/conda/lib/python3.7/site-packages/imdb/parser/http/piculet.py:262: using lxml


In [2]:
# kdramas = pd.read_csv("../input/ii-completing-dataset-with-imdbpy/kdrama_v1.csv")
kdramas = pd.read_csv("../input/imdb-korea/kdrama_v1.csv")
kdramas.shape
# kdramas.head()

(1234, 11)

In [3]:
kdramas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1234 non-null   object 
 1   year        1234 non-null   object 
 2   movie_id    1216 non-null   float64
 3   title_new   1216 non-null   object 
 4   country     1162 non-null   object 
 5   cast_IDs    1184 non-null   object 
 6   cast_names  1184 non-null   object 
 7   genres      1191 non-null   object 
 8   rating      1067 non-null   float64
 9   year_new    1199 non-null   float64
 10  type        1199 non-null   object 
dtypes: float64(3), object(8)
memory usage: 106.2+ KB


In [4]:
kdramas.shape

(1234, 11)

In [5]:
kdramas['movie_id'] = kdramas['movie_id'].fillna(-1).astype(int)
kdramas['year_new'] = kdramas['year_new'].fillna(-1).astype(int)

## I. Recheck

In [6]:
# Remove entries with non-korean origin
kdramas  = kdramas[kdramas['country'] == 'kr']

# Remove entries without casts
kdramas = kdramas[kdramas['cast_names'].notnull()]

# Limit only tv series
kdramas = kdramas[kdramas['type'] == 'tv series']

In [7]:
kdramas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1010 entries, 0 to 1233
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1010 non-null   object 
 1   year        1010 non-null   object 
 2   movie_id    1010 non-null   int64  
 3   title_new   1010 non-null   object 
 4   country     1010 non-null   object 
 5   cast_IDs    1010 non-null   object 
 6   cast_names  1010 non-null   object 
 7   genres      1006 non-null   object 
 8   rating      935 non-null    float64
 9   year_new    1010 non-null   int64  
 10  type        1010 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 94.7+ KB


In [8]:
genres = []
actorFeatures = {}
kdramas['genres'] = kdramas['genres'].fillna('unk')
# Removed unfound /unrelated genres
kdramas = kdramas[(~kdramas['genres'].str.contains("Reality-TV")) & (~kdramas['genres'].str.contains("unk"))]

for idx,row in kdramas.iterrows():
    
    # collecting genres
    gr = row['genres']
    for g in gr.split(','):
        if g not in genres:
            genres.append(g)
    
    # making new dataset: actor ID & Names
    actor_ids = row['cast_IDs'].split(',')
    actor_names = row['cast_names'].split(',')
    
    for i in range(len(actor_ids)):
        if actor_ids[i] not in actorFeatures:
            actorFeatures[actor_ids[i]] = actor_names[i]
            
actorFeatures = pd.DataFrame.from_dict(actorFeatures, orient='index').reset_index()
actorFeatures.columns = ['actor_ID', 'actor_name']
actorFeatures['actor_ID'] = actorFeatures['actor_ID'].astype(int)

In [9]:
# Make a dictionary to store averaged rating per genre
genres_dict = {}
for genre in genres:
    genres_dict[genre] = []
    
rating_filled = kdramas[kdramas.rating.notnull()]

for i, dr in rating_filled.iterrows():
    gr = dr['genres']
    for g in gr.split(','):
        genres_dict[g].append(dr['rating'])
        
for k in genres_dict.keys():
    genres_dict[k] = np.average(genres_dict[k])
    
# Fill unknown rating per genre
for i, d in kdramas.iterrows():
    if np.isnan(d['rating']):
        genres1 = d['genres'].split(',')
        genres_rating = []
        for g in genres1:
            genres_rating.append(genres_dict[g])
        kdramas.loc[i, 'rating'] = np.average(genres_rating)

In [10]:
cols = ['movie_id', 'title_new', 'cast_IDs', 'cast_names', 'genres', 'rating', 'year_new']
kdramas = kdramas[cols]
kdramas.columns=['movie_id', 'title', 'cast_IDs', 'cast_names', 'genres', 'rating','year']
# getting rid of duplicates
# kdramas = kdramas.drop_duplicates(subset = 'movie_id')

In [11]:
kdramas.head()

,movie_id,title,cast_IDs,cast_names,genres,rating,year
0,8199972,100 Days My Prince,"6765553,2305471,1443295,3498542,9170918,789033...","Kyung-soo Do,Nam Ji-Hyun,Jo Sung-ha,Jo Han-chu...","Action,Comedy,History,Romance",7.7,2018
1,5476252,12 Years Promise,"10213650,1452474,1224530,0046276,6857341,37533...","Im Ji-Hyun,So-yeon Lee,Min Namkoong,Chong-ok B...","Comedy,Drama,Romance",7.4,2014
2,12846096,18 Again,"6327190,2341244,9946376,7591921,11794543,69165...","Ha-neul Kim,Yoon Sang-Hyun,Lee Do-Hyun,No Jeon...","Comedy,Fantasy,Romance",8.2,2020
3,12015466,365: Repeat the Year,"3789806,2305471,1790874,11794543,12016813,1052...","Lee Joon-hyuk,Nam Ji-Hyun,Kim Ji-Soo,Ryeoun,Le...","Crime,Drama,Fantasy,Mystery,Thriller",8.1,2020
4,4449190,The Legendary Witch,"1623740,5754288,1085809,5204923,2291768,218888...","Ji-hye Han,Yeon-Soo Ha,Du-shim Ko,Hyun-Kyung O...","Drama,Romance",7.6,2014


In [12]:
kdramas.shape

(1005, 7)

## II. Create New Features

In [13]:
actorFeatures = actorFeatures.set_index('actor_ID')

## Assigning new Features per Genres
for g in genres:
    
    actorFeatures[g + "_count"] = 0
    actorFeatures[g + "_totalRating"] = 0
    actorFeatures[g + "_avgRating"] = 0
    actorFeatures[g +'_leadCount'] = 0
    actorFeatures[g +'_secCount'] = 0
    actorFeatures[g +'_thirdCount'] = 0
    
actorFeatures['total_drama'] = 0
actorFeatures['total_rating'] = 0
actorFeatures['recency'] = 0
actorFeatures['leadTotal'] = 0
actorFeatures['secTotal'] = 0
actorFeatures['thirdTotal'] = 0
actorFeatures['leadWt'] = 0
actorFeatures['secWt'] = 0
actorFeatures['thirdWt'] = 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

In [14]:
cr_year = 2021
# Iterating through each dramas
for i, d in kdramas.iterrows():

    cast_IDs = d['cast_IDs'].split(',')
    grs = d['genres'].split(',')
    rating = d['rating']
    
    diff_yr = cr_year - d['year'] + 1
#     TmWt = min(0.8 ** diff_yr, 0.8)

    #Iterating through each cast member
    for i in range(len(cast_IDs)):
        id = int(cast_IDs[i])
        # assigning total drama
        actorFeatures.loc[id, 'total_drama'] += 1
        # assigning sum of ratings
        actorFeatures.loc[id, 'total_rating'] += rating
        actorFeatures.loc[id, 'recency'] += diff_yr
        
        # Defining weights for leads
        if i >= 0 and i <= 2:
            actorFeatures.loc[id, 'leadTotal'] += 1
            actorFeatures.loc[id, 'leadWt'] += 3
        elif i>=3 and i <= 9:
            actorFeatures.loc[id, 'secTotal'] += 1
            actorFeatures.loc[id, 'secWt'] += 2
        else:
            actorFeatures.loc[id, 'thirdTotal'] += 1
            actorFeatures.loc[id, 'thirdWt'] += 1
        
        for g in grs:
            # assigning genre count
            actorFeatures.loc[id, g + '_count'] += 1
            # assigning genre rating sum
            actorFeatures.loc[id, g + '_totalRating'] += rating
            if i >= 0 and i <= 2:
                actorFeatures.loc[id, g + '_leadCount'] += 3
            elif i>=3 and i <= 9:
                actorFeatures.loc[id, g + '_secCount'] += 2
            else:
                actorFeatures.loc[id, g + '_thirdCount'] += 1

In [15]:
actorFeatures['recency_avg']= actorFeatures['recency']/actorFeatures['total_drama']
# Get average rating per genre
for g in genres:
    actorFeatures[g + "_avgRating"] = actorFeatures[g + "_totalRating"]/actorFeatures[g + "_count"]
    
# Fill NaNs due to 0/0
actorFeatures = actorFeatures.fillna(0)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  """Entry point for launching an IPython kernel.


In [16]:
# Select Columns
cols_count = [ g + '_count' for g in genres] 
cols_avgRating = [g + '_avgRating' for g in genres] 
cols_all = [ g + '_count' for g in genres] + [g + '_avgRating' for g in genres]

cols_lead = actorFeatures.columns[actorFeatures.columns.str.contains("Count")]
cols_posAcc = ['leadTotal', 'secTotal','thirdTotal']
cols_posWt = ['leadWt', 'secWt','thirdWt']

In [17]:
# Normalizing Genre count by dividing it by total drama per actor
for c in cols_count:
    actorFeatures[c] = actorFeatures[c]/ actorFeatures['total_drama']
    
# Normalizing Lead count by dividing it by total drama per actor
for c in cols_lead:
    actorFeatures[c] = actorFeatures[c]/ actorFeatures['total_drama']
    
# Normalizing Acc Lead count by dividing it by total drama per actor
for c in cols_posAcc:
    actorFeatures[c] = actorFeatures[c]/ actorFeatures['total_drama']
    
for c in cols_posWt:
    actorFeatures[c] = actorFeatures[c]/ actorFeatures['total_drama']
    
actorFeatures['avgRating'] = actorFeatures['total_rating']/actorFeatures['total_drama']

scaler = MinMaxScaler()
actorFeatures[['recency_avg', 'total_drama']] = scaler.fit_transform(actorFeatures[['recency_avg', 'total_drama']])
actorFeatures[cols_avgRating] = scaler.fit_transform(actorFeatures[cols_avgRating])
actorFeatures[cols_posWt] = scaler.fit_transform(actorFeatures[cols_posWt])

## III.A. Find Closest Actors by Cosine Similarity

In [18]:
# Define what features to use
cols_all = cols_posWt + cols_avgRating + ['recency_avg']

# Calculate Cosine Similarity among actors
sim = pairwise.cosine_similarity(actorFeatures[cols_all])

In [19]:
selectActor = "Kim Seon-Ho" #define which actor to assess
K = 10 #define how many actors are closest to

# Get row number of selected Actor
row_actor = actorFeatures.index.get_loc(actorFeatures.loc[actorFeatures['actor_name'].str.contains(selectActor)].index[0])
# Get K closest actorFeatures (returned row numbers)
row_cands = list(reversed(np.argsort(sim[row_actor])[-K-1:-1]))
# Show K closest actorFeatures
for i in range(K):
    print(i+1, actorFeatures.iloc[row_cands[i], 0], np.round(sim[row_actor][row_cands[i]],4))

1 Jin-young Jung 0.9874
2 Jin Yi-Han 0.9855
3 Ki Do-Hoon 0.9732
4 Oh-jung Kwon 0.9715
5 Yun-a Oh 0.9394
6 Joon-seok Heo 0.9385
7 Hye Eun Lee 0.9274
8 Sol-bin Ahn 0.9269
9 Lee Yoo-Bi 0.9269
10 Lee Pil-mo 0.9261


## Find Closest Actors by K Nearest Neighbors

In [20]:
# Find the actor ID
selectActor = "Kim Seon-Ho"
K = 5
cols_all = cols_posWt + cols_avgRating + ['recency_avg']
id_ = actorFeatures.loc[actorFeatures['actor_name'].str.contains(selectActor)].index[0]

print("Actor Name:", selectActor)
print("With K-Nearest Neighbors")

actor_vec = actorFeatures.loc[actorFeatures.index == id_, cols_all].values.reshape(1, -1) #Get the actor vector
recs = actorFeatures.drop(actorFeatures[actorFeatures.index == id_].index, axis = 0)[cols_all] #Get the rest vectors

# Define model, using K with Euclidean distance
knn = NearestNeighbors(n_neighbors=K, p=2)
knn.fit(recs)

# Return closest actorFeatures
idxs = knn.kneighbors(actor_vec, return_distance=True)
for i in range(K):
    print(i+1, actorFeatures.iloc[idxs[1][0][i]]['actor_name'], np.round(idxs[0][0][i],4))

Actor Name: Kim Seon-Ho
With K-Nearest Neighbors
1 Kang Ha-Neul 0.3749
2 Jeong-an Chae 0.3965
3 Park Joon-geum 0.5606
4 Park Hae-joon 0.5701
5 Jang Young-hyeon 0.8351
